In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

import numpy as np
import time
import datetime

In [3]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [4]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/67/mjxp58z56yj83_1x0372gkwr0000gn/T/tmp1wyy691t
INFO:vega_sim.null_service:Launching GraphQL node at port 52009
INFO:vega_sim.null_service:Launching Console at port 52015
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


#### Create the wallets

In [5]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the asset

In [6]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

In [7]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.providers.fee.distributionTimeStep", "60s")
vega.wait_for_datanode_sync()
vega.update_network_parameter(MM_WALLET.name, "market.fee.factors.infrastructureFee", "0.0")

In [8]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [9]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

In [10]:
market_decimals = 3 
position_decimals = 2

#### Create the market and set up liquidity

In [11]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=position_decimals,
        market_decimals=market_decimals,
    )

In [12]:
market_id = vega.all_markets()[0].id
print("Market id = " + market_id)

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


Market id = 0eb503c479b1b9b6841b8a7025157cdd9750aebfb765bc0025fea1e1a02aca4b


In [13]:
cumulative_sum_after_mint = 0.0 
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    cumulative_sum_after_mint +=  general + margin + bond 

print("cumulative sum = " + str(cumulative_sum_after_mint))


cumulative sum = 102000000.0


In [14]:
MM_commintment_amount=10000
vega.submit_liquidity(
        MM_WALLET.name,
        market_id=market_id,
        commitment_amount=MM_commintment_amount,
        fee=0.05,
        buy_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1), 
            ("PEGGED_REFERENCE_BEST_BID", 0.4, 1)],
        sell_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1),
            ("PEGGED_REFERENCE_BEST_ASK", 0.4, 1),
        ],
        is_amendment=True,
    )
vega.forward('1s')
vega.wait_for_datanode_sync()



In [15]:
# Submit orders which will stay on the book and create best bid / ask 
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=96,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=104,
    wait=False,
)



#### Do some trading

In [16]:
# Do a trade which will cross
price = 99
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=False,
)
vega.forward('1s')

vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=False,
)
vega.forward('1s')


# We will now have mark price of 100 


In [17]:
vega.wait_for_datanode_sync()


market_data = vega.market_data(market_id=market_id)
markprice = float(market_data.mark_price) / 10**market_decimals
print("Mark price is "+str(markprice))

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)
print("Market undistributed liquidity fees: " + str(liq_fees))

Using function with raw data from data-node VegaService.market_data. Be wary if prices/positions are not converted from int form


Mark price is 100.5
Market undistributed liquidity fees: 5.02503


In [18]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = 95, market_id=market_id)

In [21]:
vega.forward('1s')
vega.wait_for_datanode_sync()

liq_fees = vega.market_account(market_id,vega_protos.vega.ACCOUNT_TYPE_FEES_LIQUIDITY)

cumulative_sum = 0.0 
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    cumulative_sum +=  general + margin + bond 
    
print("Market undistributed liquidity fees: " + str(liq_fees))
print("cumulative sum = " + str(cumulative_sum))
print("initially minted = " + str(cumulative_sum_after_mint))
print("cumulative sum + undistributed liq fees = " + str(cumulative_sum + liq_fees)) 

if (np.abs(liq_fees - 5.02503) > 1e-3):
    raise Exception("Expected to see 5.02503 - this isn't correct but it's the bug I am expecting to demonstrate")

Market undistributed liquidity fees: 5.02503
cumulative sum = 101999994.97497
initially minted = 102000000.0
cumulative sum + undistributed liq fees = 102000000.0


In [20]:
print("Check out http://localhost:"+str(vega.data_node_graphql_port))
print("Market id = " + market_id)

#vega.stop()

Check out http://localhost:52009
Market id = 0eb503c479b1b9b6841b8a7025157cdd9750aebfb765bc0025fea1e1a02aca4b
